## DL2 to DL3

In this notebook, we will combine the DL2 *.h5* files with the instrument response function (IRF) to achieve the DL3 with magic-cta-pipe (MCP). We will use a small data sample due to time constraints.

Let's start importing some basic libraries:

In [1]:
import glob
import logging
import os
import subprocess
import sys
log = logging.getLogger()

### Processing DL2 to DL3

Here we run a script that:
1) produces DL3 data (i.e.: DL2 data after gammaness and theta cuts). To get the cut values for every run, it interpolates the cut tables produced with the MC test sample, using the run average pointing coordinates.
2) interpolates IRFs (actually in this tutorial we don't interpolate, but we show you how to do it). It also interpolates the effective area values calculated with the MC test samples for every run using its average pointing coordinates.

The DL3 and IRFs are packed together in the output files (*.fits.gz*). 

In the configuration file, we set:

1) `interpolation_method: "nearest"`, where we use the nearest MC point to estimate the IRF of our target. Ideally, we should use the other options, i.e. "linear", where we do a linear interpolation of 3 MC points around our target, or "cubic", where we do a cubic interpolation instead.
2) Maximum angular distance for interpolation.
3) Target name or coordinates.

Let's take a look at the configuration file section suited for this analysis:

In [2]:
%%capture
os.system("sed -n '260,266p' /fefs/aswg/workspace/analysis-school-2024/MCP/dl3/conf/config.yaml")

dl2_to_dl3:
    interpolation_method: "nearest"  # select "nearest", "linear" or "cubic"
    interpolation_scheme: "cosZd" # select "cosZdAz" or "cosZd"
    max_distance: "45. deg"  # angle type Quantity, or comment out to remove the cut
    source_name: "Crab"
    source_ra: null  # used when the source name cannot be resolved
    source_dec: null  # used when the source name cannot be resolved

Let's setup the configuration and data paths:

In [13]:
# The output directory where the DL3 data will be saved:
#dir_dl3 = '/fefs/aswg/workspace/analysis-school-2024/MCP/dl3/output'
dir_dl3='/fefs/aswg/workspace/YOUR_WORKSPACE/DL3/output'
f=open(f'{dir_dl3}/DL3.log','w')

# The input directory with the IRF:
dir_irf='/fefs/aswg/workspace/analysis-school-2024/MCP/irf/output'

# The input directory with the DL2 files:
input_data=('/fefs/aswg/workspace/analysis-school-2024/MCP/dl3/input/*')
input_file_data = glob.glob(input_data)
input_file_data.sort()

# The configuration file directory:
config=('/fefs/aswg/workspace/analysis-school-2024/MCP/dl3/conf/config.yaml')

Here we use subprocess to run the function that converts DL2 to DL3 and get a log file:

In [14]:
for input_file in input_file_data:  
    y = subprocess.run(['lst1_magic_dl2_to_dl3', f'-d{input_file}', f'-i{dir_irf}',\
         f'-o{dir_dl3}', f'-c{config}'], stdout=f, stderr=f)        

### Creating Index files

Then we run the command below to create **Header Data Unit (HDU)** and **Observation** index files, needed by Gammapy to process DL3 data. Also these files are saved in the DL3 output directory.

In [20]:
subprocess.run(['create_dl3_index_files', f'-i{dir_dl3}'],stdout=f, stderr=f);    

In [21]:
more $dir_dl3/DL3.log

In [23]:
ls

DL2_to_DL3.ipynb  SED_and_LC.ipynb
